**About** : This notebook is used to infer models.

In [ ]:
# %load_ext nb_black
%load_ext autoreload
%autoreload 2

In [ ]:
cd ../src/

## Initialization

### Imports

In [ ]:
import os
import torch

print(torch.__version__)
os.environ['CUDA_VISIBLE_DEVICES'] = "1"
torch.cuda.get_device_name(0)

In [ ]:
import os
import re
import cv2
import sys
import glob
import json
import time
import torch
import warnings
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

from tqdm import tqdm
from sklearn.metrics import *

warnings.simplefilter(action="ignore", category=FutureWarning)
warnings.simplefilter(action="ignore", category=UserWarning)

In [ ]:
from params import *
from data.preparation import *
from utils.plots import plot_sample, plot_sample_with_edges

In [ ]:
df = prepare_data(DATA_PATH, "")

In [ ]:
ROWS_PER_FRAME = 543  # number of landmarks per frame

def load_relevant_data_subset(pq_path):
    df = pd.read_parquet(pq_path)
    n_frames = int(len(df) / ROWS_PER_FRAME)
    data = df[['x', 'y', 'z']].values.reshape(n_frames, ROWS_PER_FRAME, 3)
    return df, data.astype(np.float32)

### Landmarks

In [ ]:
KEPT_LANDMARKS = [
    [468, 469, 470, 471, 472, 473, 474, 475, 476, 477, 478, 479, 480, 481, 482, 483, 484, 485, 486, 487, 488],  # left hand
    [522, 523, 524, 525, 526, 527, 528, 529, 530, 531, 532, 533, 534, 535, 536, 537, 538, 539, 540, 541, 542],  # right hand
    [10, 54, 67, 132, 150, 152, 162, 172, 176, 234, 284, 297, 361, 379, 389, 397, 400, 454],  # silhouette
    [13, 37, 40, 61, 78, 81, 84, 87, 88, 91, 191, 267, 270, 291, 308, 311, 314, 317, 318, 321, 415],  # lips
    [500, 501, 502, 503, 504, 505, 506, 507, 508, 509, 510, 511], # arms
    [205, 425],  # cheeks
]
MAPPING = [i + 1 for i in range(len(KEPT_LANDMARKS))]

TO_AVG = [
    [466, 387, 385, 398, 263, 390, 374, 381, 362],  # left_eye
    [246, 160, 158, 173, 33, 163, 145, 154, 133],
    [383, 293, 296, 285],  # left_eyebrow
    [156, 63, 66, 55],  # right_eyebrow
    [1, 2, 98, 327, 168],  # nose
]

In [ ]:
landmarks = np.concatenate(KEPT_LANDMARKS)
type_embed = np.zeros(1000)
start = 0
for subset, idx in zip(KEPT_LANDMARKS, MAPPING):
    print(subset, idx)
    type_embed[start: start + len(subset)] = idx
    start += len(subset)

type_embed = type_embed[type_embed > 0]

type_embed = np.concatenate([type_embed, np.array([idx] * len(TO_AVG))])

print("\nn_landmarks :", len(type_embed))

### Preprocessing class

In [ ]:
import torch
import torch.nn as nn
import torch.nn.functional as F


class Preprocessing(nn.Module):
    def __init__(self, type_embed, max_len=50):
        super(Preprocessing, self).__init__()

        self.type_embed = torch.from_numpy(type_embed[None, :].astype(np.float32))
        self.type_embed = self.type_embed.repeat(1000, 1)

        self.landmark_embed = torch.tensor(np.arange(120)).float().unsqueeze(0) + 1
        self.landmark_embed = self.landmark_embed.repeat(1000, 1)
        
        self.ids = torch.from_numpy(np.concatenate(KEPT_LANDMARKS))

        self.to_avg = [torch.tensor(avg) for avg in TO_AVG]

        self.hands = torch.tensor(
            [468, 469, 470, 471, 472, 473, 474, 475, 476, 477, 478, 479, 480, 481, 482, 483, 484, 485, 486, 487, 488] + 
            [522, 523, 524, 525, 526, 527, 528, 529, 530, 531, 532, 533, 534, 535, 536, 537, 538, 539, 540, 541, 542]
        )

        self.frames = torch.tensor(np.arange(1000) + 1)
        
        self.max_len = torch.tensor([max_len])
        self.min_1 = torch.tensor([1])
        
        self.shoulders = torch.where((self.ids == 500) | (self.ids == 501))[0]  # torch.tensor([62, 63])  # [500, 501]

    def filter_sign(self, x):
        hands = x[:, self.hands, 0]
        nan_prop = torch.isnan(hands).float().mean(-1)
        
        x = x[torch.where(nan_prop < 1)[0]]
#         if len(torch.where(nan_prop < 1)[0]) > 5:  # NEW
#             x = x[torch.where(nan_prop < 1)[0]]
#         else:
#             print(len(x), len(torch.where(nan_prop < 1)[0]))

        length = self.frames[:x.size(0)].max().unsqueeze(0)
        sz = torch.cat([length, self.max_len]).max()
        
        divisor = (((sz - self.max_len) > 0) * (sz / self.max_len) + 1).int()
        ids = (self.frames[:x.size(0)] % divisor) == 0
        
        return x[ids]
    
    def normalize_shoulders(self, x):
        shoulders = x.transpose(0, 1)[self.shoulders]  # 2 x n_frames x 3
        center = ((shoulders[0] + shoulders[1]) / 2).mean(0)
        dist = torch.sqrt(((shoulders[0] - shoulders[1]) ** 2).sum(-1)).mean()
#         print(center, dist)
        return (x - center) / (dist + 1e-6)
        
    def forward(self, x):
#         shape = x.shape
        
        x = self.filter_sign(x)
        
#         if x.shape[0] <= 3:
#             print(shape, x_.shape)
# #         else:
#         return 
            
        n_frames = x.shape[0]     
        
        avg_ids = []
        for ids in self.to_avg:
            avg_id = x[:, ids].mean(1, keepdims=True)
            avg_ids.append(avg_id)

        x = torch.cat([x[:, self.ids]] + avg_ids, 1)

        type_embed = self.type_embed[:n_frames]
        landmark_embed = self.landmark_embed[:n_frames, :x.shape[1]]
        
        # Normalize & fill nans
#         x = self.normalize_shoulders(x)
        nonan = x[~torch.isnan(x)].view(-1, x.shape[-1])
        x = x - nonan.mean(0)[None, None, :]
        x = x / nonan.std(0, unbiased=False)[None, None, :]
        x[torch.isnan(x)] = 0

        # Concat
        x = torch.cat([
            type_embed.unsqueeze(-1).to(x.device), x # , landmark_embed.unsqueeze(-1)
        ], -1).transpose(1, 2)

# #         Resize
#         x = x.contiguous().view(x.size(0), -1).transpose(0, 1)
#         x = F.interpolate(x.unsqueeze(1), 50).squeeze(1)
#         x = x.transpose(0, 1).contiguous().view(50, 5, -1)
        
        # Crop
#         x = x[:self.max_len]
        
        return x

### Main

In [ ]:
prepro = Preprocessing(type_embed, max_len=25)

In [ ]:
SAVE = False

SAVE_FOLDER = "../input/torch_17/"
os.makedirs(SAVE_FOLDER, exist_ok=True)

In [ ]:
for i in tqdm(range(len(df['path']))):
# for i in range(100):
    path = df['path'][i]
    name = f"{path.split('/')[-2]}_{path.split('/')[-1].split('.')[0]}.npy"
    
    if SAVE and os.path.exists(SAVE_FOLDER + name):
        continue

    pq, data = load_relevant_data_subset(path)
    
    data = torch.from_numpy(data).cuda()
    
    out_torch = prepro(data).cpu()

    if SAVE:
        np.save(SAVE_FOLDER + name, out_torch.numpy())

    if not (i % 30000):
        data = {
            "x": out_torch[:, 1],
            "y": out_torch[:, 2],
            "z": out_torch[:, 3],
            "type": out_torch[:, 0],
        }
        plot_sample_with_edges(data, n_frames=4, figsize=(10, 10))

    break

### Graph & flip

In [ ]:
GRAPH = [
    [42, 53, 52, 56, 59, 54, 57, 55, 58, 47],  # left head
    [42, 44, 43, 48, 51, 45, 49, 46, 50, 47],  # right head

    [63, 62, 61, 71, 72, 73, 79, 76, 66, 69, 63], # outter lips
    [64, 70, 65, 60, 75, 80, 74, 78, 77, 67, 68, 64],  # inner lips

    # RIGHT HAND
    [41, 40, 39, 38],
    [37, 36, 35, 34],
    [33, 32, 31, 30],
    [29, 28, 27, 26],
    [25, 24, 23, 22, 21],
    [38, 34, 30, 26, 21, 38],

    # LEFT HAND
    [20, 19, 18, 17],
    [16, 15, 14, 13],
    [12, 11, 10, 9],
    [8, 7, 6, 5],
    [4, 3, 2, 1, 0],
    [17, 13, 9, 5, 0, 17],
    
    # Arms
    [82, 81, 83, 85, 87, 89, 91, 85],
    [82, 84, 86, 88, 90, 92, 86],
]

In [ ]:
FLIP_MAP = {  # torch_7
    53: 44, 52: 43, 56: 48, 59: 51, 54: 45, 57: 49, 55: 46, 58: 50, 47: 47,  # head
    41: 20, 40: 19, 39: 18, 38: 17, 37: 16, 36: 15, 35: 14, 34: 13, 33: 12, 32: 11, 31: 10, 30: 9, 29: 8, 28: 7, 27: 6, 26: 5, 25: 4, 24: 3, 23: 2, 22: 1, 21: 0,  # hands
    97: 98, 95: 96, 94: 93,  # eyebrows, cheeks, eyes
    63: 73, 62: 72, 61: 71, 71: 61, 72: 62, 73: 63, 79: 69, 76: 66, 66: 76, 69: 79,  # outter lips
    64: 74, 70: 80, 65: 75, 78: 68, 67: 77,  # inner lips
    81: 82, 83: 84, 85: 86, 87: 88, 89: 90, 91: 92,  # arms
}


FLIP_ARRAY = np.arange(np.max(np.concatenate(list(FLIP_MAP.items()))) + 1)
for k, v in FLIP_MAP.items():
    FLIP_ARRAY[k] = v
    FLIP_ARRAY[v] = k

    
def flip(data, flip_array=FLIP_ARRAY, p=1):
    if np.random.random() > p:
        return data

    flip_array_ = np.arange(len(data['x'].T))
    flip_array = flip_array[:len(flip_array_)]
    flip_array_[:len(flip_array)] = flip_array

    data['x'] = - data['x']
    for k in ['x', 'y', 'z']:
        data[k] = data[k].T[flip_array_].T
    return data

In [ ]:
plot_sample_with_edges(data, n_frames=1, figsize=(10, 10), show_text=True, graph=GRAPH)

In [ ]:
data = flip(data)

In [ ]:
plot_sample_with_edges(data, n_frames=1, figsize=(10, 10), show_text=True, graph=GRAPH)

Done ! 